<a href="https://colab.research.google.com/github/a82-abbasi/image-processing/blob/main/percentage_pearlite_ferrite_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/tensorflow/examples.git

  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/seg_mod_1/

/content/drive/MyDrive/seg_mod_1


In [ ]:
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
from pathlib import Path
import re
from skimage import measure
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import matplotlib as mpl
import tqdm
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

In [ ]:

images_path = "./train/micro_project/img_256"
masks_path = "./train/micro_project/img_msmask256"

In [ ]:
def standardize(x):
    x = np.array(x, dtype='float64')
    x -= np.min(x)
    x /= np.percentile(x, 98)
    x[x > 1] = 1
    return x

def preprocessing(img):
    image = np.array(img)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = np.zeros_like(image)
    image[:,:,0] = gray
    image[:,:,1] = gray
    image[:,:,2] = gray
    image = standardize(image)
    return image

In [ ]:
images_paths = []
masks_paths = []

for imgname in os.listdir(images_path):
  images_paths.append(os.path.join(images_path,imgname))

for imgname in os.listdir(masks_path):
  masks_paths.append(os.path.join(masks_path,imgname))



In [ ]:
images_paths.sort()
masks_paths.sort()

In [ ]:


#Capture training image info as a list
train_images = []
train_masks = []

for imgpath in tqdm.tqdm(images_paths):
    img = cv2.imread(imgpath)
    img = preprocessing(img)
    train_images.append(img)


for maskpath in tqdm.tqdm(masks_paths):
     mask0 = cv2.imread(maskpath, 0)
     mask0 = np.where(mask0 > 125, 1, 0)
     train_masks.append(mask0)

train_images = np.array(train_images)
train_masks = np.array(train_masks)

X_train, X_val, y_train, y_val = train_test_split(train_images, train_masks, test_size = 0.10, shuffle=True, random_state = 1)
print("Class values: ", np.unique(y_train))

100%|██████████| 504/504 [07:15<00:00,  1.16it/s]


Class values:  [0 1]


In [ ]:
# Print shapes of arrays
print("x_train shape:", y_train.shape)
print("y_train shape:", y_val.shape)



x_train shape: (453, 256, 256)
y_train shape: (51, 256, 256)


In [ ]:
y_train=np.expand_dims(np.array(y_train), axis=3)
y_val=np.expand_dims(np.array(y_val), axis=3)

In [ ]:
NORM = mpl.colors.Normalize(vmin=0, vmax=2)

plt.figure(figsize=(16,10))
for i in range(1,4):
    plt.subplot(2,3,i)
    img = X_val[i]
    plt.imshow(img)
    plt.colorbar()
    plt.axis('off')

for i in range(4,7):
    plt.subplot(2,3,i)
    img = np.squeeze(y_val[i-3])
    plt.imshow(img, cmap='jet', norm=NORM)
    plt.colorbar()
    plt.axis('off')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt

# تابع اصلاح‌شده برای IOU (برای segmentation باینری)
def binary_iou(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # تبدیل y_true به float32
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # threshold و تبدیل به float32
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3]) - intersection
    iou = (intersection + 1e-7) / (union + 1e-7)  # جلوگیری از تقسیم بر صفر
    return tf.reduce_mean(iou)

# تابع برای محاسبه IOU تک تصویر در show_predictions
def single_image_iou(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # تبدیل y_true به float32
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true + y_pred) - intersection
    return (intersection + 1e-7) / (union + 1e-7)

def unet_model(output_channels):
    IMG_HEIGHT = X_train.shape[1]
    IMG_WIDTH  = X_train.shape[2]
    IMG_CHANNELS = X_train.shape[3]

    base_model = MobileNetV2(input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS], include_top=False)

    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]

    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
    down_stack = Model(inputs=base_model.input, outputs=base_model_outputs)
    down_stack.trainable = False

    up_stack = [
        pix2pix.upsample(512, 3),  # 4x4 -> 8x8
        pix2pix.upsample(256, 3),  # 8x8 -> 16x16
        pix2pix.upsample(128, 3),  # 16x16 -> 32x32
        pix2pix.upsample(64, 3),   # 32x32 -> 64x64
    ]

    inputs = Input(shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = Concatenate()
        x = concat([x, skip])

    last = Conv2DTranspose(output_channels, 3, strides=2, padding='same', activation='sigmoid')
    x = last(x)
    return Model(inputs=inputs, outputs=x)

def create_mask(pred_mask):
    pred_mask = tf.sigmoid(pred_mask)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

def show_predictions(epoch, dataset=None, num=50):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            iou_score = single_image_iou(mask[0], create_mask(pred_mask))
            plt.figure(figsize=(15, 10))
            plt.subplot(231)
            plt.title('Testing Image')
            plt.imshow(image[0], cmap='gray')
            plt.subplot(232)
            plt.title('Ground Truth')
            plt.imshow(mask[0], cmap='gray')
            plt.subplot(233)
            plt.title(f'Prediction on test image\nIOU: {iou_score:.4f}')
            plt.imshow(create_mask(pred_mask), cmap='gray')

            plt.savefig(f"results/mask_{str(ii)}.png")
            plt.show()
            print(f'IOU Score for this image: {iou_score:.4f}')
    else:
        fig = plt.figure(figsize=(12, 12))
        fig.suptitle(f"\n Epoch: {str(epoch)}\n", fontsize=16)

        pred_mask = model.predict(train_images[num][tf.newaxis, ...])
        iou_score = single_image_iou(train_masks[num], create_mask(pred_mask))
        plt.subplot(331)
        plt.title('Testing Image')
        plt.imshow(train_images[num], cmap='gray')
        plt.subplot(332)
        plt.title('Ground Truth')
        plt.imshow(train_masks[num], cmap='gray')
        plt.subplot(333)
        plt.title(f'Prediction on test image\nIOU: {iou_score:.4f}')
        plt.imshow(create_mask(pred_mask)[:,:,0], cmap='gray')

        pred_mask = model.predict(train_images[num+16][tf.newaxis, ...])
        iou_score = single_image_iou(train_masks[num+16], create_mask(pred_mask))
        plt.subplot(334)
        plt.imshow(train_images[num+16], cmap='gray')
        plt.subplot(335)
        plt.imshow(train_masks[num+16], cmap='gray')
        plt.subplot(336)
        plt.title(f'Prediction on test image\nIOU: {iou_score:.4f}')
        plt.imshow(create_mask(pred_mask)[:,:,0], cmap='gray')

        pred_mask = model.predict(train_images[num+14][tf.newaxis, ...])
        iou_score = single_image_iou(train_masks[num+14], create_mask(pred_mask))
        plt.subplot(337)
        plt.imshow(train_images[num+14], cmap='gray')
        plt.subplot(338)
        plt.imshow(train_masks[num+14], cmap='gray')
        plt.subplot(339)
        plt.title(f'Prediction on test image\nIOU: {iou_score:.4f}')
        plt.imshow(create_mask(pred_mask)[:,:,0], cmap='gray')

        plt.show()
        print(f'IOU Scores for displayed images: {iou_score:.4f} (last image)')

output_channels = 1
model = unet_model(output_channels)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', binary_iou])

EPOCHS = 50
VAL_SUBSPLITS = 5
BATCH_SIZE = 16
VALIDATION_STEPS = len(X_val)//BATCH_SIZE//VAL_SUBSPLITS
STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE

class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        show_predictions(epoch)
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

model_history = model.fit(X_train, y_train, epochs=EPOCHS,
                         batch_size=BATCH_SIZE,
                         verbose=1,
                         validation_data=(X_val, y_val),
                         callbacks=[DisplayCallback()])

/tmp/ipython-input-3968630675.py:29: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS], include_top=False)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step


ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 

<Figure size 1200x1200 with 0 Axes>

In [ ]:
def iou_metric(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # threshold 0.5
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + 1e-7)


In [ ]:
def unet_model1(output_channels):
    IMG_HEIGHT = X_train.shape[1]
    IMG_WIDTH  = X_train.shape[2]
    IMG_CHANNELS = X_train.shape[3]

    base_model = MobileNetV2(input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS], include_top=False, weights='imagenet')

    # Use the activations of these layers
    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]

    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

    # Create the feature extraction model
    down_stack = Model(inputs=base_model.input, outputs=base_model_outputs)

    down_stack.trainable = False

    up_stack = [
        pix2pix.upsample(512, 3),  # 4x4 -> 8x8
        pix2pix.upsample(256, 3),  # 8x8 -> 16x16
        pix2pix.upsample(128, 3),  # 16x16 -> 32x32
        pix2pix.upsample(64, 3),   # 32x32 -> 64x64
    ]

    inputs = Input(shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])

    # Downsampling through the model
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = Concatenate()
        x = concat([x, skip])

    # This is the last layer of the model
    last = Conv2DTranspose(output_channels, 3, strides=2, padding='same', activation='sigmoid')  #64x64 -> 128x128

    x = last(x)

    return Model(inputs=inputs, outputs=x)

In [ ]:
def unet_model(output_channels):
    IMG_HEIGHT = X_train.shape[1]
    IMG_WIDTH  = X_train.shape[2]
    IMG_CHANNELS = X_train.shape[3]

    base_model = MobileNetV2(input_shape=[256, 256, 3], include_top=False)


    # Use the activations of these layers
    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]

    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

    # Create the feature extraction model
    down_stack = Model(inputs=base_model.input, outputs=base_model_outputs)

    down_stack.trainable = False

    up_stack = [
        pix2pix.upsample(512, 3),  # 4x4 -> 8x8
        pix2pix.upsample(256, 3),  # 8x8 -> 16x16
        pix2pix.upsample(128, 3),  # 16x16 -> 32x32
        pix2pix.upsample(64, 3),   # 32x32 -> 64x64
    ]

    inputs = Input(shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])

    # Downsampling through the model
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = Concatenate()
        x = concat([x, skip])

    # This is the last layer of the model
    last = Conv2DTranspose(output_channels, 3, strides=2, padding='same', activation='sigmoid')  #64x64 -> 128x128

    x = last(x)

    return Model(inputs=inputs, outputs=x)

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.sigmoid(pred_mask)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


def show_predictions(epoch, dataset=None, num=50):
  if dataset:

    for image, mask in dataset.take(num):
        pred_mask = model.predict(image)
        pred_mask_bin = tf.cast(create_mask(pred_mask) > 0.5, tf.int32)
        mask_bin = tf.cast(mask[0], tf.int32)
        iou_val = np.sum(pred_mask_bin * mask_bin) / (np.sum(pred_mask_bin) + np.sum(mask_bin) - np.sum(pred_mask_bin * mask_bin) + 1e-7)
        print(f"IOU for this image: {iou_val:.4f}")
        plt.figure(figsize=(15, 10))
        plt.subplot(231)
        plt.title('Testing Image')
        plt.imshow(image[0], cmap='gray')
        plt.subplot(232)
        plt.title('Ground Truth')
        plt.imshow(mask[0], cmap='gray')
        plt.subplot(233)
        plt.title('Prediction on test image')
        plt.imshow(create_mask(pred_mask), cmap='gray')

        plt.savefig(f"results/mask_{str(ii)}.png")

        plt.show()

  else:
      fig = plt.figure(figsize=(12, 12))
      fig.suptitle(f"\n Epoch: {str(epoch)}\n", fontsize=16)

      plt.subplot(331)
      plt.title('Testing Image')
      plt.imshow(train_images[num], cmap='gray')
      plt.subplot(332)
      plt.title('Ground Truth')
      plt.imshow(train_masks[num], cmap='gray')
      plt.subplot(333)
      plt.title('Prediction on test image')
      plt.imshow(create_mask(model.predict(train_images[num][tf.newaxis, ...]))[:,:,0], cmap='gray')

      plt.subplot(334)
      plt.imshow(train_images[num+16], cmap='gray')
      plt.subplot(335)
      plt.imshow(train_masks[num+16], cmap='gray')
      plt.subplot(336)
      plt.imshow(create_mask(model.predict(train_images[num+16][tf.newaxis, ...]))[:,:,0], cmap='gray')
      plt.subplot(337)
      plt.imshow(train_images[num+14], cmap='gray')
      plt.subplot(338)
      plt.imshow(train_masks[num+14], cmap='gray')
      plt.subplot(339)
      plt.imshow(create_mask(model.predict(train_images[num+14][tf.newaxis, ...]))[:,:,0], cmap='gray')

      #plt.savefig(f"results/mask_{str(num+100)}_{str(epoch)}.png")

      plt.show()

In [ ]:
output_channels = 1  # یا تعداد کانال‌های خروجی مورد نظر را اینجا مشخص کنید
model = unet_model1(output_channels)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', iou_metric])


/tmp/ipython-input-3954321839.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS], include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
EPOCHS = 50
VAL_SUBSPLITS = 5
BATCH_SIZE = 16
VALIDATION_STEPS = len(X_val)//BATCH_SIZE//VAL_SUBSPLITS
STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
# sample_image = train_images[0]
# sample_mask = train_masks[0]

class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        show_predictions(epoch)
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

model_history = model.fit(X_train, y_train, epochs=EPOCHS,
                           batch_size = BATCH_SIZE,
                          verbose=1,
                          #steps_per_epoch=STEPS_PER_EPOCH,
                          #validation_steps=VALIDATION_STEPS,
                          validation_data=(X_val, y_val),
                          callbacks=[DisplayCallback()]
                          )


In [ ]:
import matplotlib.pyplot as plt

# ---- Plot Loss ----
plt.figure(figsize=(8,6))
plt.plot(model_history.history['loss'], label='Training Loss')
plt.plot(model_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()

# ---- Plot Accuracy ----
plt.figure(figsize=(8,6))
plt.plot(model_history.history['accuracy'], label='Training Accuracy')
plt.plot(model_history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.show()


In [ ]:
import json

# ذخیره history بعد از آموزش
with open('/content/drive/MyDrive/seg_mod_1/training_history.json', 'w') as f:
    json.dump(model_history.history, f)


In [ ]:
import json
import matplotlib.pyplot as plt

# بارگذاری history
with open('/content/drive/MyDrive/seg_mod_1/training_history.json', 'r') as f:
    history = json.load(f)

# رسم تغییرات IOU
plt.figure(figsize=(8,5))
plt.plot(history['iou_metric'], label='Training IOU')
plt.plot(history['val_iou_metric'], label='Validation IOU')
plt.title('IOU Metric Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.grid(True)
plt.show()
